In [1]:
from src.countries_hook import CountriesHook

hook = CountriesHook()
hook.parse_countries()
hook.parse_tables()


In [3]:
hook.timezones.query('common_name == "Brazil"')

,common_name,timezones
0,Brazil,UTC-05:00
1,Brazil,UTC-04:00
2,Brazil,UTC-03:00
3,Brazil,UTC-02:00
